<a href="https://colab.research.google.com/github/danielbrunolabs/supply_chain/blob/main/supply_chain_analysis_with_knowledge_graphs_and_genai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
from google.colab import userdata

NEO4J_URI = userdata.get('NEO4J_URI') #https://console.neo4j.io/
NEO4J_USER_NAME = userdata.get('NEO4J_USER_NAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')
AZURE_OPEN_AI_KEY = userdata.get('AZURE_OPEN_AI_KEY')
AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')

os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPEN_AI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT

In [25]:
%%capture
%pip install pandas py2neo langchain langchain-openai langchain_community neo4j

In [12]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [26]:
graph = Neo4jGraph(
    url = NEO4J_URI,
    username = NEO4J_USER_NAME,
    password = NEO4J_PASSWORD
)

chain = GraphCypherQAChain.from_llm(
    AzureChatOpenAI(temperature=0,
                    azure_deployment="gpt-4o",
                    api_version="2024-02-15-preview",
                    max_tokens=None,
                    timeout=None,
                    max_retries=2),
    allow_dangerous_requests = True,
    graph = graph,
    verbose = True
)

In [27]:
# MATCH path=(:Product)<-[:SUPPLIES]-(c:Supplier)
# where c.country = "Finland"
# RETURN path;
# Returns: Karkki supplies 3 products: Maxilaku, Valkoinen suklaa, Lakkalikööri

chain.invoke({"query": "Which products are supplied by Karkki Oy?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (s:Supplier {companyName: "Karkki Oy"})-[:SUPPLIES]->(p:Product)
RETURN p.productID, p.productName

Full Context:
[{'p.productID': '49', 'p.productName': 'Maxilaku'}, {'p.productID': '50', 'p.productName': 'Valkoinen suklaa'}, {'p.productID': '76', 'p.productName': 'Lakkalikööri'}]

> Finished chain.


{'query': 'Which products are supplied by Karkki Oy?',
 'result': 'Maxilaku, Valkoinen suklaa, Lakkalikööri are supplied by Karkki Oy.'}